In [1]:
from crvusdsim.pool import get

ETH_POOL = "weth"
sim_market = get(ETH_POOL, bands_data="controller")
pool = sim_market.pool
stableswap_pools = sim_market.stableswap_pools

In [2]:
# Verify snapshot context works on LLAMMA pools
i, j = 0, 1
high = pool.get_max_trade_size(i, j)

prev_bals = (sum(pool.bands_x.values()), sum(pool.bands_y.values()))

print("Pre Trade")
print("Balances", prev_bals)
print("High", high)
print("Stablecoin", pool.BORROWED_TOKEN.balanceOf[pool.address])
print("Collateral", pool.COLLATERAL_TOKEN.balanceOf[pool.address])
print()

with pool.use_snapshot_context():
    amt_in, amt_out, _ = pool.trade(i, j, high)
    swap_bals = (sum(pool.bands_x.values()), sum(pool.bands_y.values()))
    print("Amount in", high)
    print("Amount out", amt_out)
    print()
    print("Post Trade")
    print("Balances", swap_bals)
    print("Stablecoin", pool.BORROWED_TOKEN.balanceOf[pool.address])
    print("Collateral", pool.COLLATERAL_TOKEN.balanceOf[pool.address])
    print()

new_bals = (sum(pool.bands_x.values()), sum(pool.bands_y.values()))
print("After reversing snapshot context")
print("Balances", new_bals)
print("Stablecoin", pool.BORROWED_TOKEN.balanceOf[pool.address])
print("Collateral", pool.COLLATERAL_TOKEN.balanceOf[pool.address])

assert prev_bals == new_bals, (prev_bals, new_bals)
assert swap_bals != new_bals

Pre Trade
Balances (0, 15971445880068407183360)
High 40119568578427971526152911
Stablecoin 0
Collateral 15971445880068407183360

Amount in 40119568578427971526152911
Amount out 12702926360139514396671

Post Trade
Balances (40119568578427971525912220, 3268519519928892786689)
Stablecoin 40119568578427971526152911
Collateral 3268519519928892786689

After reversing snapshot context
Balances (0, 15971445880068407183360)
Stablecoin 0
Collateral 15971445880068407183360


In [3]:
# Verify snapshot context works on StableSwap pools from crvusdsim
i, j = 1, 0

stableswap_pool = stableswap_pools[2]
high = stableswap_pool.get_max_trade_size(i, j)
prev_bals = stableswap_pool.balances.copy()

print("Pre Trade")
print("Balances", prev_bals)
print("High", high)
print("Allowance", stableswap_pool.coins[0].balanceOf[stableswap_pool.address])
print()

with stableswap_pool.use_snapshot_context():
    amt_out = stableswap_pool.trade(i, j, high)
    swap_bals = stableswap_pool.balances.copy()
    print("Post Trade")
    print("Amount out", amt_out)
    print("Balances", swap_bals)
    print("Allowance", stableswap_pool.coins[0].balanceOf[stableswap_pool.address])
    print()

new_bals = stableswap_pool.balances.copy()
print("After reversing snapshot context")
print("Balances", new_bals)
print("Allowance", stableswap_pool.coins[0].balanceOf[stableswap_pool.address])

assert prev_bals == new_bals, (prev_bals, new_bals)
assert swap_bals != new_bals

Pre Trade
Balances [10008658850325, 25873939907962300163632541]
High 12842547157360454826337406
Allowance 10008658850325

Post Trade
Amount out (12842547157360454826337406, 9907581404595, 990857226182175000850)
Balances [100582017117, 38716487065322754989969947]
Allowance 101077445730

After reversing snapshot context
Balances [10008658850325, 25873939907962300163632541]
Allowance 10008658850325
